<a href="https://colab.research.google.com/github/TitapornEve/twitter_Crawler/blob/main/%E0%B8%95%E0%B8%B1%E0%B8%94%E0%B8%84%E0%B8%B3%E0%B8%AB%E0%B8%A2%E0%B8%B2%E0%B8%9A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pythainlp
!pip install emoji

     |████████████████████████████████| 10.6MB 4.4MB/s 
     |████████████████████████████████| 747kB 22.6MB/s 
     |████████████████████████████████| 133kB 5.5MB/s 


In [3]:
from nltk import NaiveBayesClassifier as nbc
from sklearn.feature_extraction.text import CountVectorizer

from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords

import codecs

from itertools import chain

import pandas as pd
import numpy as np

import emoji
import re

from wordcloud import WordCloud
import matplotlib.pyplot as plt 

In [11]:
tweet=pd.read_excel("twitterCrawler55.xlsx")


In [ ]:
tweet.info()

In [ ]:
tweet.describe()

In [ ]:
tweet.head()

In [ ]:
tweet.drop_duplicates("text")

In [ ]:
tweet.text

In [17]:
with codecs.open('pos.txt', 'r', "utf-8") as f:
    lines = f.readlines()
listpos=[e.strip() for e in lines]
del lines
f.close() # ปิดไฟล์

In [19]:
with codecs.open('neg.txt', 'r', "utf-8") as f:
    lines = f.readlines()
listneg=[e.strip() for e in lines]
f.close() # ปิดไฟล์

In [ ]:
pos1=['ไม่หยาบ']*len(listpos)
neg1=['หยาบ']*len(listneg)

data = list(zip(listpos,pos1)) + list(zip(listneg,neg1))

vocabulary = set(chain(*[word_tokenize(i[0].lower()) for i in data]))


feature_set = [({i:(i in word_tokenize(sentence.lower())) for i in vocabulary},tag) for sentence, tag in data]

classifier = nbc.train(feature_set)

#while True:
for x in tweet.text:
  test_sentence= x
  featurized_test_sentence={i:(i in word_tokenize(test_sentence.lower()))for i in vocabulary}
  print("test_sent:",test_sentence)
  print("ตัดคำ:",word_tokenize(test_sentence))
  print("tag:",classifier.classify(featurized_test_sentence))
  print("\n")